# Pandas

Материалы:
* Макрушин С.В. "Лекция 2: Библиотека Pandas"
* https://pandas.pydata.org/docs/user_guide/index.html#
* https://pandas.pydata.org/docs/reference/index.html
* Уэс Маккини. Python и анализ данных

## Лабораторная работа №2

### Базовые операции с `DataFrame`

1.1 В файлах `recipes_sample.csv` и `reviews_sample.csv` находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца с индексами в таблице `reviews` (безымянный столбец).

In [ ]:
import pandas as pd

recipes = pd.read_csv('recipes_sample.csv')

reviews = pd.read_csv('reviews_sample.csv', index_col=0)

print (recipes)
print (reviews)

1.2 Для каждой из таблиц выведите основные параметры:
* количество точек данных (строк);
* количество столбцов;
* тип данных каждого столбца.

In [ ]:
print("recipes")
print("строк", recipes.shape[0])
print("столбцов", recipes.shape[1])
print(recipes.dtypes)
print()
print("reviews")
print("строк", reviews.shape[0])
print("столбцов", reviews.shape[1])
print(reviews.dtypes)

1.3 Исследуйте, в каких столбцах таблиц содержатся пропуски. Посчитайте долю строк, содержащих пропуски, в отношении к общему количеству строк.

In [ ]:

recipes_rows_with_missings = recipes.isna().any(axis=1).sum()
recipes_percentage_missings = (recipes_rows_with_missings / recipes.shape[0]) * 100

print("recipes :", recipes_percentage_missings, "%")

reviews_rows_with_missings = reviews.isna().any(axis=1).sum()
recipes_percentage_missings =  (reviews_rows_with_missings / reviews.shape[0]) * 100

print("reviews :", recipes_percentage_missings, "%")

1.4 Рассчитайте среднее значение для каждого из числовых столбцов (где это имеет смысл).

In [ ]:
recipes_column = recipes.select_dtypes(include=['int64', 'float64'])
recipes_mean = recipes_column.mean()
print("recipes: ")
print(recipes_mean)

reviews_column = reviews.select_dtypes(include=['int64', 'float64'])
reviews_mean = reviews_column.mean()
print("reviews: ")
print(reviews_mean)

1.5 Создайте серию из 10 случайных названий рецептов.

In [ ]:
recipes_rand = recipes.sample(10)['name']
recipes_rand

1.6 Измените индекс в таблице `reviews`, пронумеровав строки, начиная с нуля.

In [ ]:
new_index = pd.RangeIndex(start=0, stop=len(reviews))

reviews.index = new_index

print(reviews)

1.7 Выведите информацию о рецептах, время выполнения которых не больше 20 минут и кол-во ингредиентов в которых не больше 5.

In [ ]:
filt = (recipes['minutes'] <= 20) & (recipes['n_ingredients'] <= 5)
rec = recipes[filt]
print(rec)

### Работа с датами в `pandas`

2.1 Преобразуйте столбец `submitted` из таблицы `recipes` в формат времени. Модифицируйте решение задачи 1.1 так, чтобы считать столбец сразу в нужном формате.

In [ ]:
recipes = pd.read_csv('recipes_sample.csv', parse_dates=['submitted'])
print(recipes['submitted'])

2.2 Выведите информацию о рецептах, добавленных в датасет не позже 2010 года.

In [ ]:
filt = (recipes['submitted'].dt.year <= 2010)
rec = recipes[filt]
print(rec)

### Работа со строковыми данными в `pandas`

3.1  Добавьте в таблицу `recipes` столбец `description_length`, в котором хранится длина описания рецепта из столбца `description`.

In [ ]:
recipes['description_length'] = recipes['description'].apply(lambda x: len(str(x)))
print(recipes)

3.2 Измените название каждого рецепта в таблице `recipes` таким образом, чтобы каждое слово в названии начиналось с прописной буквы.

In [ ]:
recipes['name'] = recipes['name'].str.title()
print(recipes)

3.3 Добавьте в таблицу `recipes` столбец `name_word_count`, в котором хранится количество слов из названии рецепта (считайте, что слова в названии разделяются только пробелами). Обратите внимание, что между словами может располагаться несколько пробелов подряд.

In [ ]:
recipes['name_word_count'] = recipes['name'].apply(lambda x: len(str(x).split()))
print(recipes)

### Группировки таблиц `pd.DataFrame`

4.1 Посчитайте количество рецептов, представленных каждым из участников (`contributor_id`). Какой участник добавил максимальное кол-во рецептов?

In [ ]:
rec = recipes.groupby('contributor_id')['name'].count()
print(rec)
print(rec.idxmax(), rec.max())

4.2 Посчитайте средний рейтинг к каждому из рецептов. Для скольких рецептов отсутствуют отзывы? Обратите внимание, что отзыв с нулевым рейтингом или не заполненным текстовым описанием не считается отсутствующим.

In [ ]:
rec = pd.read_csv('recipes_sample.csv')
rev = pd.read_csv('reviews_sample.csv', index_col=0)
rat = rev.groupby('recipe_id')['rating'].mean()
rat1 = set(rec['id'])
rat2 = set(rev['recipe_id'])
count = rat1-rat2
print(rat)
print(len(count))

4.3 Посчитайте количество рецептов с разбивкой по годам создания.

In [ ]:
rec = pd.read_csv('recipes_sample.csv', parse_dates=['submitted'])
year = rec['submitted'].dt.year
year.value_counts()

### Объединение таблиц `pd.DataFrame`

5.1 При помощи объединения таблиц, создайте `DataFrame`, состоящий из четырех столбцов: `id`, `name`, `user_id`, `rating`. Рецепты, на которые не оставлен ни один отзыв, должны отсутствовать в полученной таблице. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и попытавшись найти строку, соответствующую этому рецепту, в полученном `DataFrame`.

In [ ]:
merge = pd.merge(rec[['id', 'name']], rev[['recipe_id', 'user_id', 'rating']], left_on='id', right_on='recipe_id', how='inner')

miss_rev = set(rec['id'])-set(rev['recipe_id'])

check = merge[merge['id'] == list(miss_rev)[0]]

print(merge)
print(check)

5.2 При помощи объединения таблиц и группировок, создайте `DataFrame`, состоящий из трех столбцов: `recipe_id`, `name`, `review_count`, где столбец `review_count` содержит кол-во отзывов, оставленных на рецепт `recipe_id`. У рецептов, на которые не оставлен ни один отзыв, в столбце `review_count` должен быть указан 0. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и найдя строку, соответствующую этому рецепту, в полученном `DataFrame`.

In [ ]:
rev['review_count'] = rev.groupby('recipe_id').count()['review']
rev = rev[['recipe_id', 'review_count']].fillna(0).join(rec['name'])
rev

5.3. Выясните, рецепты, добавленные в каком году, имеют наименьший средний рейтинг?

In [ ]:
mean = reviews.groupby('recipe_id')['rating'].mean()
min = recipes[recipes['id'] == mean.idxmin()]
min

### Сохранение таблиц `pd.DataFrame`

6.1 Отсортируйте таблицу в порядке убывания величины столбца `name_word_count` и сохраните результаты выполнения заданий 3.1-3.3 в csv файл. 

In [ ]:
recipes_sort = recipes.sort_values('name_word_count', ascending=False)
recipes_sort.to_csv('recipes_sorted1.csv', index=False)

6.2 Воспользовавшись `pd.ExcelWriter`, cохраните результаты 5.1 и 5.2 в файл: на лист с названием `Рецепты с оценками` сохраните результаты выполнения 5.1; на лист с названием `Количество отзывов по рецептам` сохраните результаты выполнения 5.2.

In [ ]:
with pd.ExcelWriter('results.xlsx') as writer:

    merge.to_excel(writer, sheet_name='Рецепты с оценками', index=False)
    
    rev.to_excel(writer, sheet_name='Количество отзывов по рецептам', index=False)

#### [версия 2]
* Уточнены формулировки задач 1.1, 3.3, 4.2, 5.1, 5.2, 5.3